<a href="https://colab.research.google.com/github/Screwygosling/UnsupervisedLearning/blob/main/STL10Unsupervised.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("jessicali9530/stl10")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'stl10' dataset.
Path to dataset files: /kaggle/input/stl10


Initialize dataset


In [ ]:
from torchvision import datasets, transforms
import os
import shutil

In [ ]:
base = path
# Directory where actual unlabeled images are located
unlabeled_images_dir = os.path.join(base, "unlabeled_images")

# Change target to a writable directory like /content/unknown
target = os.path.join("/content", "unknown")

os.makedirs(target, exist_ok=True)

# Check if unlabeled_images_dir exists and contains files
if os.path.exists(unlabeled_images_dir) and os.path.isdir(unlabeled_images_dir):
    # Get a list of files to move to avoid issues if directory changes during iteration
    files_to_move = list(os.listdir(unlabeled_images_dir))
    for filename in files_to_move:
        if filename.lower().endswith(("png", "jpg", "jpeg")):
            source_path = os.path.join(unlabeled_images_dir, filename)
            destination_path = os.path.join(target, filename)
            shutil.copy2(source_path, destination_path)

    # The original directory is on a read-only filesystem, so it cannot be removed.
    # The following block is commented out because os.rmdir will fail due to read-only filesystem.
    # if not os.listdir(unlabeled_images_dir): # This check ensures it's empty
    #     os.rmdir(unlabeled_images_dir)

KeyboardInterrupt: 

In [ ]:
transform = transforms.ToTensor()

unlabeled_data = datasets.ImageFolder(
    root=base,
    transform=transform
)

print(len(unlabeled_data))
img, label = unlabeled_data[0]
print(img.shape, label)


In [ ]:
from torchvision import transforms
from torchvision.datasets import ImageFolder

In [ ]:
transform = transforms.Compose([
    transforms.Resize((96, 96)),
    transforms.ToTensor()
])

dataset = ImageFolder(root=base, transform=transform)

In [ ]:
print(len(dataset))
img, label = dataset[0]
print(img.shape, label)

In [ ]:
from torch.utils.data import DataLoader

dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

In [ ]:
import torch
import torch.nn as nn
from torchvision.models import resnet

In [ ]:
class Autoencoder(nn.Module):
    def __init__(self):
        super().__init__()

        self.encoder = nn.Sequential(
            nn.Conv2d(3, 32, 3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 64, 3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 128, 3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(128, 256, 3, stride=2, padding=1),
            nn.ReLU()
        )

        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(256, 128, 3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, 3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 32, 3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(32, 3, 3, stride=2, padding=1, output_padding=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

In [ ]:
# initialize training

model = Autoencoder()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

In [ ]:
# start training

epochs = 10

for epoch in range(epochs):
    model.train()
    total_loss = 0

    for imgs, _ in dataloader:
        imgs = imgs.to(device)

        outputs = model(imgs)
        loss = criterion(outputs, imgs)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss = {total_loss/len(dataloader):.4f}")

In [ ]:
model.eval()

embeddings = []

with torch.no_grad():
    for imgs, _ in dataloader:
        imgs = imgs.to(device)
        latent = model.encoder(imgs)
        latent = latent.view(latent.size(0), -1)
        embeddings.append(latent.cpu())

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
kmeans = KMeans(n_clusters=10)
cluster_ids = kmeans.fit_predict(embeddings)

Kmeans is used to classify the images
